In [1]:
from funcoes_auxiliares import *

from sklearn.linear_model import LinearRegression

pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_rows = 6

In [2]:
lista_estados = ["AC", "AL", "AM", "AP", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RO", "RS", "RR", "SC", "SE", "SP", "TO"]

In [3]:
df_estados_brasil = None

In [4]:
for i, estado in enumerate(lista_estados):
    nome_arquivo_despesas = "../dados-limpos/aquisicao_por_classe_de_rendimento_e_estado/%s/dados-limpos/%s_tipos_despesas.csv" % (estado, estado)
    df_despesas = carregar_df_despesas_gerais(nome_arquivo_despesas)
    df_invertido = df_despesas_gerais_invertido(df_despesas, estado, False)
    if(i == 0):
        df_estados_brasil = df_invertido
    else:
        df_estados_brasil = pd.concat([df_estados_brasil, df_invertido])

In [5]:
df_estados_brasil.reset_index(drop=True, inplace=True)

In [6]:
df_estados_brasil

,Alimentação,Habitação,Aluguel,Aluguel monetário,Aluguel não monetário,Condomínio,Serviços e taxas,Energia elétrica,Telefone fixo,Telefone Celular,...,Imóvel (aquisição),Imóvel (reforma),Outros investimentos,Diminuição do passivo,Empréstimo,Prestação de imóvel,Tamanho médio da família,Total gasto por classes,Estado,Classe social
0,592.63,730.80,376.63,17.01,359.62,1.17,182.51,79.36,2.87,31.01,...,9.44,47.66,0.00,59.75,57.38,2.38,3.72,4763.74,AC,Até 1908 (2)
1,767.97,1107.24,557.82,29.06,528.76,0.59,297.58,124.18,1.92,49.11,...,17.81,105.58,0.00,202.88,199.49,3.39,4.07,7809.43,AC,Mais de 1908 a 2862
2,822.06,1645.56,828.81,29.04,799.78,4.72,420.86,167.51,4.09,71.63,...,246.55,211.45,0.00,534.44,502.56,31.88,3.99,13290.87,AC,Mais de 2862 a 5724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,697.99,1558.67,766.06,127.11,638.95,0.00,491.32,214.92,10.12,21.08,...,6.58,17.79,2.92,301.25,221.85,79.40,3.48,12985.20,TO,Mais de 5724 a 9540
160,770.28,2207.58,838.66,66.28,772.37,0.00,557.28,261.26,1.53,38.42,...,0.00,23.89,0.00,223.34,223.34,0.00,3.43,20047.08,TO,Mais de 9540 a 14310
161,709.70,3165.85,1128.58,163.28,965.30,0.00,614.35,294.36,0.00,21.92,...,885.46,557.83,0.00,1546.38,1059.03,487.34,3.01,28066.60,TO,Mais de 14310 a 23850


In [7]:
classes_sociais = ["Até 1908 (2)", "Mais de 1908 a 2862", "Mais de 2862 a 5724", "Mais de 5724 a 9540", "Mais de 9540 a 14310", "Mais de 14310 a 23850"]

## Análise de regressão linear

### Até 1908 (2)

In [8]:
lista_colunas_afetam_valor_total = df_estados_brasil.columns[:-3]

X = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[0]][lista_colunas_afetam_valor_total]
Y = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[0]]["Total gasto por classes"]

lm = LinearRegression()
lm.fit(X, Y)

r2 = lm.score(X.values, Y)

print("Coeficiente estimado:", lm.coef_)
print("\nR2 (score):", r2)

print("\nIntercept: %.5f" % lm.intercept_)

equacao_str = ""
for coef, feat in zip(lm.coef_,lista_colunas_afetam_valor_total):
    equacao_str += " + (%.5f * '%s')" % (coef, feat)

print("\nEquação do 'valor-mensal' em termos de todos os atributos analisados:\n\n  f(x) = %.5f%s" % (lm.intercept_, equacao_str))

print("\nCoeficiente de determinação é: %.5f, ou seja, %.5f%% do 'valor-mensal' não é explicada pelos atributos utilizados" % (r2, (1-r2)*100))

Coeficiente estimado: [ 6.78122108e-01  2.59020118e+00  1.96320541e+00 -1.34671831e+00
 -2.17943210e+00  9.99190450e-02  2.09170200e+00 -5.95638599e-01
  8.58355268e-02 -3.08621447e-01 -5.95304119e-01  1.10634652e-01
 -8.90099159e-01 -4.65864545e-01  4.57157639e-01  3.04300105e-01
  9.59809967e-01 -2.06361386e-01 -7.50979074e-03  1.84996827e+00
  8.57889338e-01  9.51448854e-01  2.60127154e-01  5.98502958e-01
  1.39615313e-01  1.81899214e-02  2.26505233e+00  4.13013233e-01
  5.66458714e-01 -8.21438150e-02 -4.81763518e-01 -3.36710356e-01
  9.01815978e-02  5.99417099e-02  2.49905023e-01  3.51871099e-01
  9.90784193e-02  9.41543702e-02  1.84145948e-01  3.35666811e-01
  1.61581346e+00  1.58561129e+00  3.98326629e-01  9.81960753e-02
  7.31474651e-01  1.86032209e+00 -4.82279348e-03  3.67642443e-01
  3.80860227e-01  3.11751549e-01  1.62818441e-01  4.47892545e-01
  5.03956416e-02 -4.71372763e-02  6.39152774e-02  5.99417099e-02
  2.49905023e-01  3.51871099e-01  9.90784193e-02  9.41543702e-02
  1

### Mais de 1908 a 2862

In [9]:
lista_colunas_afetam_valor_total = df_estados_brasil.columns[:-3]

X = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[1]][lista_colunas_afetam_valor_total]
Y = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[1]]["Total gasto por classes"]

lm = LinearRegression()
lm.fit(X, Y)

r2 = lm.score(X.values, Y)

print("Coeficiente estimado:", lm.coef_)
print("\nR2 (score):", r2)

print("\nIntercept: %.5f" % lm.intercept_)

equacao_str = ""
for coef, feat in zip(lm.coef_,lista_colunas_afetam_valor_total):
    equacao_str += " + (%.5f * '%s')" % (coef, feat)

print("\nEquação do 'valor-mensal' em termos de todos os atributos analisados:\n\n  f(x) = %.5f%s" % (lm.intercept_, equacao_str))

print("\nCoeficiente de determinação é: %.5f, ou seja, %.5f%% do 'valor-mensal' não é explicada pelos atributos utilizados" % (r2, (1-r2)*100))

Coeficiente estimado: [ 1.27970088e+00 -4.12361913e-01  2.70865532e+00  1.37568954e+00
  8.39348809e-01  4.37085393e-01  2.63585766e+00 -2.32805542e-01
 -9.79921667e-02  2.88104855e-01  5.68441651e-01 -6.21822358e-01
 -3.76541610e-01  3.74081177e-01  1.94951258e+00  2.84372129e-01
  1.28867061e+00  9.38351276e-01  6.79278197e-02  1.73781131e+00
  2.33249844e-01  1.00231352e+00  3.78920261e-01  3.58586637e-01
  1.06412062e-01  1.32904331e-01  1.08802388e+00  2.63886656e-01
  6.77900959e-01  7.26584408e-01  5.89152324e-01  9.18860118e-01
  2.86676058e-01  1.15028624e-01 -1.63927973e-01 -1.57892284e-01
  1.63624233e-01  1.97301079e-01  2.39029416e-01  3.22920765e-03
  8.42830443e-01  7.44946112e-02  1.48340189e-01 -8.53879425e-02
  1.02357423e+00  2.19021914e+00  5.88886945e-01 -8.22081769e-01
  3.18452147e-01 -1.65014306e-01 -5.29178377e-02  2.99130727e-01
 -5.18512309e-02 -1.32093113e-02  7.94930658e-02  1.15028624e-01
 -1.63927973e-01 -1.57892284e-01  1.63624233e-01  1.97301079e-01
  2

### Mais de 2862 a 5724

In [10]:
lista_colunas_afetam_valor_total = df_estados_brasil.columns[:-3]

X = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[2]][lista_colunas_afetam_valor_total]
Y = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[2]]["Total gasto por classes"]

lm = LinearRegression()
lm.fit(X, Y)

r2 = lm.score(X.values, Y)

print("Coeficiente estimado:", lm.coef_)
print("\nR2 (score):", r2)

print("\nIntercept: %.5f" % lm.intercept_)

equacao_str = ""
for coef, feat in zip(lm.coef_,lista_colunas_afetam_valor_total):
    equacao_str += " + (%.5f * '%s')" % (coef, feat)

print("\nEquação do 'valor-mensal' em termos de todos os atributos analisados:\n\n  f(x) = %.5f%s" % (lm.intercept_, equacao_str))

print("\nCoeficiente de determinação é: %.5f, ou seja, %.5f%% do 'valor-mensal' não é explicada pelos atributos utilizados" % (r2, (1-r2)*100))

Coeficiente estimado: [ 9.17278810e-01  4.42243232e-01  7.17090515e-02  2.38965483e+00
  2.12927135e+00  1.24446118e+00  1.81210404e+00  1.44772434e-01
 -7.62137195e-03 -1.11261698e-03  5.02317482e-01  2.99786659e-01
  4.73760851e-01 -5.35565048e-01  6.07770808e-01  2.40362391e-01
  2.24038906e-01  8.22260461e-01  5.88705564e-02  2.32874611e+00
  2.30373084e-01  4.77078091e-02  1.43297770e-02  4.70966189e-01
  1.62165696e-02  8.16338536e-02  2.52337977e+00  5.43360088e-01
 -6.03554838e-01  7.37106647e-01  1.42026669e-01 -5.65308568e-01
 -8.99036964e-01  2.37188584e-01 -3.11065615e-01  2.04908618e-01
  2.69784378e-01  5.14831533e-02 -4.61747957e-02  2.29043733e-02
  5.61132381e-01 -6.55748746e-01  4.49729923e-02  8.71234288e-02
  1.57895009e+00  5.08557383e-01  8.19101628e-01  7.30492533e-01
 -3.29490195e-02 -1.01559090e-01  5.31446642e-02  7.86027223e-01
  2.04069838e-01 -1.04785919e-01  2.02271537e-01  2.37188584e-01
 -3.11065615e-01  2.04908618e-01  2.69784378e-01  5.14831533e-02
 -4

### Mais de 5724 a 9540

In [11]:
lista_colunas_afetam_valor_total = df_estados_brasil.columns[:-3]

X = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[3]][lista_colunas_afetam_valor_total]
Y = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[3]]["Total gasto por classes"]

lm = LinearRegression()
lm.fit(X, Y)

r2 = lm.score(X.values, Y)

print("Coeficiente estimado:", lm.coef_)
print("\nR2 (score):", r2)

print("\nIntercept: %.5f" % lm.intercept_)

equacao_str = ""
for coef, feat in zip(lm.coef_,lista_colunas_afetam_valor_total):
    equacao_str += " + (%.5f * '%s')" % (coef, feat)

print("\nEquação do 'valor-mensal' em termos de todos os atributos analisados:\n\n  f(x) = %.5f%s" % (lm.intercept_, equacao_str))

print("\nCoeficiente de determinação é: %.5f, ou seja, %.5f%% do 'valor-mensal' não é explicada pelos atributos utilizados" % (r2, (1-r2)*100))

Coeficiente estimado: [ 2.04065307e+00  2.57616498e-01  7.79581962e+00 -2.17804175e+00
 -3.43905442e+00 -8.96003381e-01 -1.79664599e+00  1.15061719e+00
 -2.79077917e-01  5.46186719e-01 -3.88412996e-01 -4.36541605e-02
 -1.12230483e+00  8.24640657e-01  2.14142258e+00  8.92259987e-01
  3.18533302e-01  9.27697052e-01  3.27402318e-01  3.42164467e+00
  1.37793032e+00  1.19110863e+00  1.52377573e-01  1.33725112e+00
  6.75038428e-01  1.55565425e-02 -5.95445481e-02  1.38074140e+00
  7.44651840e-01  1.19172895e+00  1.09142143e+00  1.95220728e+00
  8.78094395e-01  2.38281254e-01 -5.46480261e-01  2.12966228e-01
  3.53358425e-01  1.24709401e-01 -1.24201757e+00 -1.57299395e-01
 -7.97520415e-01  1.83602606e+00 -1.35915177e-01  1.99685823e-01
  6.69286858e-01  2.07987704e+00  9.65362171e-01  6.38392659e-01
 -5.62460776e-01  1.48654582e-01 -1.72308211e-01  2.52096387e-01
 -5.50288703e-01 -3.60399239e-02  6.31724100e-02  2.38281254e-01
 -5.46480261e-01  2.12966228e-01  3.53358425e-01  1.24709401e-01
 -1

### Mais de 9540 a 14310

In [12]:
lista_colunas_afetam_valor_total = df_estados_brasil.columns[:-3]

X = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[4]][lista_colunas_afetam_valor_total]
Y = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[4]]["Total gasto por classes"]

lm = LinearRegression()
lm.fit(X, Y)

r2 = lm.score(X.values, Y)

print("Coeficiente estimado:", lm.coef_)
print("\nR2 (score):", r2)

print("\nIntercept: %.5f" % lm.intercept_)

equacao_str = ""
for coef, feat in zip(lm.coef_,lista_colunas_afetam_valor_total):
    equacao_str += " + (%.5f * '%s')" % (coef, feat)

print("\nEquação do 'valor-mensal' em termos de todos os atributos analisados:\n\n  f(x) = %.5f%s" % (lm.intercept_, equacao_str))

print("\nCoeficiente de determinação é: %.5f, ou seja, %.5f%% do 'valor-mensal' não é explicada pelos atributos utilizados" % (r2, (1-r2)*100))

Coeficiente estimado: [ 1.23382315e+00  1.93355149e+00  5.84308278e-01  1.71704319e-01
  4.12649593e-01  4.43011494e-01  2.36622235e-01 -3.61713383e-01
 -6.70704133e-02  6.04011930e-01  1.18267299e-01 -4.19701651e-02
 -2.49403144e-01  2.34506464e-01  4.88480789e-01  1.17852468e-01
  3.08796307e-01 -2.28097418e-01 -1.74159438e-02  1.54466422e+00
  1.80326960e-01  5.04092664e-01  2.43926071e-01  4.36226550e-01
  1.15249229e-01  6.48173726e-02  1.82183223e+00  2.42133905e-01
  3.39667769e-01 -4.99513550e-02  4.81080579e-01  3.47144596e-02
  7.17949176e-01  5.62955849e-02  3.58282786e-02  2.17690782e-01
 -5.85981982e-03  1.08766123e-01  1.17366048e-01  3.24222736e-02
  8.38751080e-01  9.28417211e-02  1.52109343e-01  3.00996005e-02
  5.63707581e-01  1.49135924e+00  2.98783398e-01  6.74611332e-01
  1.41374119e-01  1.34569813e-01  8.00632298e-02 -6.04611950e-02
  5.50529096e-02  1.31676238e-01 -1.66199516e-04  5.62955849e-02
  3.58282786e-02  2.17690782e-01 -5.85981982e-03  1.08766123e-01
  1

### Mais de 14310 a 23850

In [13]:
classes_sociais[4]

'Mais de 9540 a 14310'

In [14]:
lista_colunas_afetam_valor_total = df_estados_brasil.columns[:-3]

X = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[5]][lista_colunas_afetam_valor_total]
Y = df_estados_brasil[df_estados_brasil['Classe social'] == classes_sociais[5]]["Total gasto por classes"]

lm = LinearRegression()
lm.fit(X, Y)

r2 = lm.score(X.values, Y)

print("Coeficiente estimado:", lm.coef_)
print("\nR2 (score):", r2)

print("\nIntercept: %.5f" % lm.intercept_)

equacao_str = ""
for coef, feat in zip(lm.coef_,lista_colunas_afetam_valor_total):
    equacao_str += " + (%.5f * '%s')" % (coef, feat)

print("\nEquação do 'valor-mensal' em termos de todos os atributos analisados:\n\n  f(x) = %.5f%s" % (lm.intercept_, equacao_str))

print("\nCoeficiente de determinação é: %.5f, ou seja, %.5f%% do 'valor-mensal' não é explicada pelos atributos utilizados" % (r2, (1-r2)*100))

Coeficiente estimado: [ 1.07074603e-01  5.48398877e+00 -1.11260333e+00  7.85524690e-02
 -1.53982989e+00 -2.73307328e+00 -2.72101223e+00  1.49172764e-01
 -4.14475459e-01 -2.11451800e-01 -3.06048434e-01 -1.33138006e-01
 -5.30320340e-01 -3.17347904e-02 -2.40378703e+00 -6.47860735e-02
 -2.82016718e+00 -6.34735179e-01 -4.47469660e-02  1.75519311e+00
  1.38233521e+00 -1.86406845e-01  2.56132845e-01  3.62560646e-01
  2.55764076e-01  1.09947984e-01  2.21236467e+00  5.82884610e-01
  1.47057536e+00 -3.62968415e-01 -8.50766350e-01  9.01066438e-02
  4.30752425e-01 -1.19015691e-01 -8.70010239e-02  6.93699507e-01
  5.71422646e-01  9.11151338e-02  1.00540770e-01 -2.13052616e-01
  4.29917329e-01  4.35327307e-03 -2.17340432e-01 -8.71191976e-03
  1.82947500e-01  1.08664145e+00  1.39198253e-01  5.28654059e-02
 -4.36761862e-01 -1.60431626e-01  1.98949317e-01  4.53124887e-01
  1.61908986e-01  2.27104345e-02 -1.01572349e-01 -1.19015691e-01
 -8.70010239e-02  6.93699507e-01  5.71422646e-01  9.11151338e-02
  1